In [10]:
import nltk
nltk.download('brown')
nltk.download('punkt')
print(list(nltk.corpus.brown.fileids())[:3])

[nltk_data] Downloading package brown to
[nltk_data]     /home1/07144/yw23374/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home1/07144/yw23374/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['ca01', 'ca02', 'ca03']


In [61]:
import tweepy
import pandas as pd
import ast
from nltk.corpus import brown
from nltk import word_tokenize
from nltk.probability import FreqDist
import math
import warnings;
warnings.filterwarnings('ignore');

In [5]:
#Read dataframes by date
from pathlib import Path  
date_range = pd.date_range(start='2021-07-19', end='2021-08-11')

df_dict = {}
for i in date_range:
    date = i.strftime('%Y-%m-%d')
    filepath = Path('dataframes_by_date/'+date+".csv")
    df_dict[date] = pd.read_csv(filepath, encoding = "utf-8")

In [6]:
df_dict['2021-07-21']

,Unnamed: 0,id,time,text,city_name,coordinates,place
0,5753,1417635474898923529,2021-07-21 00:00:10+00:00,@KathrynTappen @NBCSports @NBCOlympics Stay sa...,"Michigan, USA","[[[-90.4181075, 41.696088], [-82.122971, 41.69...",Place(_api=<tweepy.api.API object at 0x2b23538...
1,7078,1417635772790939650,2021-07-21 00:01:21+00:00,"The Olympics are back, y’all. Let’s play some ...","Orange City, FL","[[[-81.3155653, 28.897085], [-81.26003, 28.897...",Place(_api=<tweepy.api.API object at 0x2b23538...
2,5761,1417635990026530824,2021-07-21 00:02:13+00:00,man kobe know who that is !!!! https://t.co/56...,"Terrytown, LA","[[[-90.043765, 29.878501], [-90.008313, 29.878...",Place(_api=<tweepy.api.API object at 0x2b23538...
3,2281,1417636084733956101,2021-07-21 00:02:35+00:00,SO FREAKIN EXCITING!!!! https://t.co/IAkMrJcnrZ,"Auburn, AL","[[[-85.569969, 32.532449], [-85.413112, 32.532...",Place(_api=<tweepy.api.API object at 0x2b23538...
4,4224,1417636088890408969,2021-07-21 00:02:36+00:00,First event of #TokyoOlympics 🙌🏻 https://t.co/...,"Chicago, IL","[[[-87.940033, 41.644102], [-87.523993, 41.644...",Place(_api=<tweepy.api.API object at 0x2b23538...
...,...,...,...,...,...,...,...
1666,29023,1417996975736250371,2021-07-21 23:56:38+00:00,If an athlete from china or Russia turned thei...,"Barling, AR","[[[-94.323756, 35.320062], [-94.296113, 35.320...",Place(_api=<tweepy.api.API object at 0x2b23538...
1667,13522,1417996982765957120,2021-07-21 23:56:40+00:00,See now this suspension makes sense because no...,"Pittsburgh, PA","[[[-80.095586, 40.3615796], [-79.8657933, 40.3...",Place(_api=<tweepy.api.API object at 0x2b23538...
1668,57501,1417997212769001474,2021-07-21 23:57:35+00:00,USA v Canada 🥎 lets goooo!! #olympics #usa htt...,"Fairview Park, OH","[[[-81.875476, 41.417577], [-81.815851, 41.417...",Place(_api=<tweepy.api.API object at 0x2b23538...
1669,59805,1417997419585937409,2021-07-21 23:58:24+00:00,Night two of the Olympics will get underway in...,"Catonsville, MD","[[[-76.794408, 39.22709], [-76.688592, 39.2270...",Place(_api=<tweepy.api.API object at 0x2b23538...


In [52]:
# document frequency: in how many files of the Brown corpus does each word appear
brown.fileids()
stopwords = nltk.corpus.stopwords.words("english")
stopwords += ["rt", "got", "thats", "would", "going", "u", 
               "get", "also", "one", "could", "said", "like",
               "via", "say", "amp", "many", "need", "day", "want", 
               "last", "people", "done", "two",
               "outbreak", "know", "upon", "may", "way",
               "saw", "gtgt", "olympics", "tokyo2020", "tokyo", "olympics2021", "usa"
               "olympics2020", "@olympics", "tokyoolympics", "olympic", "https"]

docfreq = { }

for brown_part in brown.fileids():
    fd = nltk.FreqDist(brown.words(fileids = brown_part))
    
    for word in fd.keys():
        word = word.lower()
        
        if word in stopwords:
            continue
        if not word.isalnum():
            continue
            
        if word in docfreq:
            docfreq[word] = docfreq[word] + 1
        else:
            docfreq[word] = 1

In [53]:

words_example = word_tokenize(df_dict['2021-07-21']["text"][0])
print(words_example)

['@', 'KathrynTappen', '@', 'NBCSports', '@', 'NBCOlympics', 'Stay', 'safe', '.', 'Man', 'I', 'hope', 'I', "'m", 'wrong', 'about', 'how', 'bad', 'this', 'could', 'get', '.', 'Love', 'watching…', 'https', ':', '//t.co/fqFC3xLko5']


In [87]:
# Use an NLTK FreqDist object to compute the frequency of each word in Frankenstein
def compute_freq_dict(sentence):
    freq_example = FreqDist()
    words = word_tokenize(sentence)
    for word in words:
        word = word.lower()

        if word in stopwords:
            continue
        if not word.isalnum():
            continue
        freq_example[word.lower()] += 1 
    return freq_example
# len(freq_example)
# freq_example
#freq_example['NBCSports']

In [88]:
def append_word_freq(date):
    tweets_list = df_dict[date]['text']
    word_freq_dict_list = []
    for sentence in tweets_list:
        word_freq_dict = compute_freq_dict(sentence)
        word_freq_dict_list.append(word_freq_dict)    
    df_dict[date]["word_freq_dict"] = word_freq_dict_list

In [89]:
#do word freqency analysis for all dates
for i in date_range:
    append_word_freq(i.strftime('%Y-%m-%d'))

In [97]:
def compute_tf_idf(sentence):
    dfidf = FreqDist()
    num_brown_parts = len(list(brown.fileids()))
    words = word_tokenize(sentence)
    tf_dict = compute_freq_dict(sentence)
    for word in words:
        word = word.lower()

        if word in stopwords:
            continue
        if not word.isalnum():
            continue
        if word not in docfreq:
            continue
        df = docfreq[word]
        idf = math.log(num_brown_parts / df )
        dfidf[word] = tf_dict[word]*idf

In [101]:
def append_tf_idf_freq(date):
    tweets_list = df_dict[date]['text']
    word_tf_idf_list = []
    for sentence in tweets_list:
        word_tf_idf = compute_tf_idf(sentence)
        word_tf_idf_list.append(word_tf_idf)    
    df_dict[date]["tf_idf"] =  word_tf_idf_list

In [102]:
#do TF-IDF analysis for all dates
for i in date_range:
    append_tf_idf_freq(i.strftime('%Y-%m-%d'))

In [103]:
df_dict['2021-07-21']

,Unnamed: 0,id,time,text,city_name,coordinates,place,word_freq_dict,tf_idf
0,5753,1417635474898923529,2021-07-21 00:00:10+00:00,@KathrynTappen @NBCSports @NBCOlympics Stay sa...,"Michigan, USA","[[[-90.4181075, 41.696088], [-82.122971, 41.69...",Place(_api=<tweepy.api.API object at 0x2b23538...,"{'kathryntappen': 1, 'nbcsports': 1, 'nbcolymp...",None
1,7078,1417635772790939650,2021-07-21 00:01:21+00:00,"The Olympics are back, y’all. Let’s play some ...","Orange City, FL","[[[-81.3155653, 28.897085], [-81.26003, 28.897...",Place(_api=<tweepy.api.API object at 0x2b23538...,"{'back': 1, 'let': 1, 'play': 1, 'softball': 1}",None
2,5761,1417635990026530824,2021-07-21 00:02:13+00:00,man kobe know who that is !!!! https://t.co/56...,"Terrytown, LA","[[[-90.043765, 29.878501], [-90.008313, 29.878...",Place(_api=<tweepy.api.API object at 0x2b23538...,"{'man': 1, 'kobe': 1}",None
3,2281,1417636084733956101,2021-07-21 00:02:35+00:00,SO FREAKIN EXCITING!!!! https://t.co/IAkMrJcnrZ,"Auburn, AL","[[[-85.569969, 32.532449], [-85.413112, 32.532...",Place(_api=<tweepy.api.API object at 0x2b23538...,"{'freakin': 1, 'exciting': 1}",None
4,4224,1417636088890408969,2021-07-21 00:02:36+00:00,First event of #TokyoOlympics 🙌🏻 https://t.co/...,"Chicago, IL","[[[-87.940033, 41.644102], [-87.523993, 41.644...",Place(_api=<tweepy.api.API object at 0x2b23538...,"{'first': 1, 'event': 1}",None
...,...,...,...,...,...,...,...,...,...
1666,29023,1417996975736250371,2021-07-21 23:56:38+00:00,If an athlete from china or Russia turned thei...,"Barling, AR","[[[-94.323756, 35.320062], [-94.296113, 35.320...",Place(_api=<tweepy.api.API object at 0x2b23538...,"{'athlete': 1, 'china': 1, 'russia': 1, 'turne...",None
1667,13522,1417996982765957120,2021-07-21 23:56:40+00:00,See now this suspension makes sense because no...,"Pittsburgh, PA","[[[-80.095586, 40.3615796], [-79.8657933, 40.3...",Place(_api=<tweepy.api.API object at 0x2b23538...,"{'see': 1, 'suspension': 1, 'makes': 1, 'sense...",None
1668,57501,1417997212769001474,2021-07-21 23:57:35+00:00,USA v Canada 🥎 lets goooo!! #olympics #usa htt...,"Fairview Park, OH","[[[-81.875476, 41.417577], [-81.815851, 41.417...",Place(_api=<tweepy.api.API object at 0x2b23538...,"{'usa': 2, 'v': 1, 'canada': 1, 'lets': 1, 'go...",None
1669,59805,1417997419585937409,2021-07-21 23:58:24+00:00,Night two of the Olympics will get underway in...,"Catonsville, MD","[[[-76.794408, 39.22709], [-76.688592, 39.2270...",Place(_api=<tweepy.api.API object at 0x2b23538...,"{'night': 1, 'underway': 1, 'five': 1, 'minute...",None


In [104]:
df_dict['2021-07-21']["word_freq_dict"][0].most_common(2)

[('kathryntappen', 1), ('nbcsports', 1)]

In [176]:
def merge_word_freq(date):
    tweets_list = df_dict[date]['text']
    word_freq_all = FreqDist()
    for sentence in tweets_list:
        word_freq = compute_freq_dict(sentence)
        word_freq_all =  word_freq_all + word_freq
    return word_freq_all

In [177]:
freq_by_date = {}
for i in date_range:
    date = i.strftime('%Y-%m-%d')
    freq_by_date[date] = {"time": date, "word_freq": merge_word_freq(date)}

In [178]:
freq_by_date['2021-08-11']['word_freq'].most_common(10)

[('gold', 6),
 ('years', 5),
 ('miss', 4),
 ('ass', 4),
 ('think', 4),
 ('watch', 4),
 ('still', 4),
 ('love', 4),
 ('sports', 3),
 ('team', 3)]

# Hot Topics:
"covid", "cancel", "softball", "openingceremony", "skateboarding", "basketball", "swimming", "simone/biles" 
"gold", "marathon", "volleyball", "closingceremony", 

In [207]:
def plot_topic_mention(topic):
    freq_list = [freq_by_date[date.strftime('%Y-%m-%d')]['word_freq'][topic] for date in date_range]
    # import plotly.express as px
    fig = px.line(x=date_range, y=freq_list, title=topic+' mention frequnecy by time'
                 ,labels=dict(x="date", y= topic+" Mention Times", color="Place"))
    fig.show()  

## Plot topics mention times by time

In [212]:
hot_topics = ["covid", "cancel", "softball", "openingceremony", "skateboarding", "basketball", 
              "swimming", "simone", "gold", "marathon", "volleyball", "closing"]
for topic in hot_topics:
    plot_topic_mention(topic)